In [1]:
# Standard library imports
import numpy as np
import pandas as pd
import ast
import os
import sys
import logging
from os.path import dirname as up

# Important directories
code_dir = os.path.dirname(os.getcwd())
deep_cal_dir = os.path.dirname(os.path.dirname(os.getcwd()))

# Allows to import my own module
sys.path.insert(0, code_dir)

from ann.model import RegressorLiftedHeston
from ann.helpers import open_data, norm, create_train_test_split
from ann.dataloader import LiftedHestonDataset
from ann.train import train

from torch.utils.data import DataLoader
from torch import nn, optim
import seaborn as sns

In [2]:
database = open_data(deep_cal_dir + '/data/heston_img.csv',info=True) 
database['vol_imp'] = database['vol_imp'].apply(ast.literal_eval)  
database['vol_imp'] = database['vol_imp'].apply(np.array)  


Five first rows of the generated DataFrame : 
           r       rho    lambda        nu     theta        v0  \
0  0.087345 -0.345281  1.797595  0.108334  0.373375  0.306434   
1  0.053734 -0.299285  0.815884  0.197408  0.314842  0.099705   
2  0.064063 -0.605398  1.594868  0.179371  0.353517  0.066062   
3  0.072809 -0.190020  1.548088  0.485789  0.139705  0.497143   
4  0.041415 -0.025787  1.500803  0.257766  0.135586  0.295568   

                                             vol_imp  
0  [2.5740481126152277, 1.2547427160366569, 0.684...  
1  [2.5740388185920797, 1.2469129301995416, 0.603...  
2  [2.574038812011191, 1.2468701153260764, 0.6083...  
3  [2.5742988249225363, 1.259290445567614, 0.6760...  
4  [2.574052452786126, 1.2501608960087736, 0.6332...  

DataFrame shape : (40000, 7)



In [3]:
train_data, train_labels, test_data, test_labels = create_train_test_split(database,0.8,1.,'vol_imp',norm=False,lifted=True,random_state = 123) 

In [4]:
train_dataset = LiftedHestonDataset(train_data,train_labels)
train_loader = DataLoader(dataset=train_dataset,
                          batch_size=32,
                          shuffle=True,
                          num_workers=1)

test_dataset = LiftedHestonDataset(test_data,test_labels)
test_loader = DataLoader(dataset=test_dataset,
                          batch_size=32,
                          shuffle=True,
                          num_workers=1)

In [5]:
model = RegressorLiftedHeston()

We can make a trial training with 100 epochs.

In [7]:
train(model,train_loader,test_loader,nb_epochs=100,seed=0,save_model_dir=code_dir + '/ann/ckpt_model',log_df= code_dir + '/ann/learning_csv/log_df.csv')

Validation loss decreased (inf --> 0.0010525922989472747).  Saving model ...
Validation loss decreased (0.0010525922989472747 --> 0.0008275320869870484).  Saving model ...
Validation loss decreased (0.0008275320869870484 --> 0.0007765439222566783).  Saving model ...
Validation loss decreased (0.0007765439222566783 --> 0.0006737885414622724).  Saving model ...
Validation loss decreased (0.0006737885414622724 --> 0.00046566451783291996).  Saving model ...
Validation loss decreased (0.00046566451783291996 --> 0.0004180661926511675).  Saving model ...
Validation loss decreased (0.0004180661926511675 --> 0.00034762316499836743).  Saving model ...
Validation loss decreased (0.00034762316499836743 --> 0.00034740340197458863).  Saving model ...
Validation loss decreased (0.00034740340197458863 --> 0.00030432670610025525).  Saving model ...
Validation loss decreased (0.00030432670610025525 --> 0.00027144557680003345).  Saving model ...
Validation loss decreased (0.00027144557680003345 --> 0.000

(    Train Losses         Test Losses  Learning Rate
 0       0.008858  tensor(4.2104e-06)          0.001
 1       0.000921  tensor(3.3101e-06)          0.001
 2       0.000777  tensor(3.1062e-06)          0.001
 3       0.000703  tensor(2.6952e-06)          0.001
 4       0.000562  tensor(1.8627e-06)          0.001
 ..           ...                 ...            ...
 95      0.000099  tensor(3.9905e-07)          0.001
 96      0.000099  tensor(4.1240e-07)          0.001
 97      0.000097  tensor(3.8592e-07)          0.001
 98      0.000097  tensor(3.7699e-07)          0.001
 99      0.000098  tensor(3.9969e-07)          0.001
 
 [100 rows x 3 columns],
 tensor(9.1584e-05))